# Pix2Pix with Tensorflow 2.0 and Kubeflow Pipelines

**WIP/TODO**

- Add link + copyright to Pix2Pix homepage + Tensorflow 2.0 Tutorials


### Requirements

**WIP/TODO**

- Setup a Kubeflow Cluster
    - Easiest : Deploy Kubeflow on GKE : Cloud Deploy 
    - Customize installation (Add NFS Storage, Google Storage, GPU node pool, "gcloud services enable ')
       - GPU require a Google Account with Billing Enabled
       - Get 300$ of FREE credits
    - Start a Jupyter Server on Kubeflow
    - Clone this project repository in your Jupyter Notebook environment


- Learn more on Kubeflow...
- Learn more on Tensorflow 2.0

## What you will learn

**WIP/TODO** 
- Use Kubelow to create a Deep Learning Pipeline for Tensorflow 2.0 (using GPU)
- Learn to create Kubeflow Pipelines Components
- Define and execute a Kubelow pipelines from this Notebook

## A Word on Pix2Pix and the Training Pipeline we will define below

**WIP/TODO**

## Get ready for the execution of this Notebook

In [1]:
# -------------------------------------
#           IMPORTANT : 
#      Customize this variable with 
#        your own GCP Project ID
#  (will be used by Google Cloud Build)
# -------------------------------------


GCP_PROJECT_ID=None

assert (GCP_PROJECT_ID) != None, "Your must set your own GCP Project ID"


In [2]:
# -------------------------------------
#    Notebook configuration 'magic'
# -------------------------------------

%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'


If necessary install the latest Kubeflow Pipelines SDK. This Python library allows to define a pipeline composed of multiple tasks, and execute it on a Kubeflow Cluster 

In [3]:
# -------------------------------------
#     Import Kubeflow Pipelines SDK 
# -------------------------------------

import kfp
import kfp.dsl as dsl
import kfp.gcp as gcp
import kfp.notebook
import kfp.components as comp
from kfp import compiler
from kubernetes import client as k8s_client

In [4]:
# -------------------------------------
#          Import PiX2Pix code 
# -------------------------------------

from download_dataset import *
from prepare_dataset import *
from train_pix2pix import *


In [5]:
# -------------------------------------
#    DEFINE SOME PROJECT VARIABLES
# -------------------------------------
URL = "https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz"
FILE_NAME = "facades.tar.gz"

NFS_MOUNT = "/mnt/nfs"
KERAS_CACHE_DIR = "/mnt/nfs/data/"

MODEL = "/mnt/nfs/data/models/"
PIX2PIX_OUTPUTS = "/mnt/nfs/data/pix2pix_outputs/"


### Learn to Create Kubeflow Pipelines Components 

There are 3 ways of using the Pix2Pix source code:

-  First, **without using Kubeflow**, we can normaly use the python code from download_dataset.py as for any other python code. For example:

````
# -------------------------------------
#       Local Python execution 
#           (No Kubeflow)
# -------------------------------------

!python download_dataset.py --fname facades.tar.gz \ 
--origin "https://people.eecs.berkeley.edu/~tinghuiz/projects/pix2pix/datasets/facades.tar.gz"
````

- We can also create a **Kubeflow pipeline component**, by simply converting a Python function into a Kubeflow Pipelines Operation, as in the code example below:

   (**However, make sure** that all the python libraries that will be used are installed in the Docker base image used) 


```
# -------------------------------------
#   Convert a Python function into a
#     Kubeflow Pipelines Operation 
# -------------------------------------

download_op = comp.func_to_container_op(download_dataset,
                                        base_image='tensorflow/tensorflow:1.14.0-py3' )

```

- Finally, we can also create a **Kubeflow pipeline component**, by packaging the Python Function in a **Docker Image**, as we are going to do in the next steps. Remember that building Docker Image can take severals minutes

  Remark: At the time of writing, there is no official Tensorflow 2.0 Docker image available yet, so we manually install Tensorflow 2.0 beta on top of a Tensorflow CUDA 10 image

In [6]:
%%bash 

# -------------------------------------
#     Define a Dockerfile, Build a  
#       Docker image to package 
#   the Kubeflow Pipelines Operation 
# -------------------------------------

cat > ./Dockerfile <<- "EOF"
FROM tensorflow/tensorflow:1.14.0-py3
RUN pip install --quiet tensorflow==2.0.0-beta1
ADD ./download_dataset.py /ml/download_dataset.py
WORKDIR /ml

ENTRYPOINT ["python", "/ml/download_dataset.py"]

EOF


You need a Docker Registry for storing the Docker image. If you are using GCP/GKE, you can use the **Google Cloud Build** tools to build the image and store them in your Google Storage : Execute the following code **AFTER having set your own GCP PROJECT ID** 

In [7]:
%%bash -s "$GCP_PROJECT_ID"

# -------------------------------------
#       Build a Docker Image on GCP 
#         using the gcloud tool
# -------------------------------------

gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}
gcloud builds submit --tag gcr.io/$1/download_dataset:latest .

----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "4b91137b-fee9-445e-bf5f-e3831947bf68"

FETCHSOURCE
Fetching storage object: gs://wl-tex10-kfp-001_cloudbuild/source/1562960926.18-17a587f6d097441cb3861cd2fe104355.tgz#1562960950097965
Copying gs://wl-tex10-kfp-001_cloudbuild/source/1562960926.18-17a587f6d097441cb3861cd2fe104355.tgz#1562960950097965...
| [1 files][178.2 MiB/178.2 MiB]                                                
Operation completed over 1 objects/178.2 MiB.                                    
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker

Step 1/5 : FROM tensorflow/tensorflow:1.14.0-py3
1.14.0-py3: Pulling from tensorflow/tensorflow
5b7339215d1d: Already exists
14ca88e9f672: Already exists
a31c3b1caad4: Already exists
b054a26005b7: Already exists
8832e3773578: Pulling fs layer
5e671b828b2a: Pulling fs layer
2b940936f993: Pulling fs layer
016724bbd2c9: Pulling fs layer
5bd1cb597025: Pulling fs layer


Activated service account credentials for: [kubeflow-user@wl-tex10-kfp-001.iam.gserviceaccount.com]
Creating temporary tarball archive of 1500 file(s) totalling 292.2 MiB before compression.
Uploading tarball of [.] to [gs://wl-tex10-kfp-001_cloudbuild/source/1562960926.18-17a587f6d097441cb3861cd2fe104355.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/wl-tex10-kfp-001/builds/4b91137b-fee9-445e-bf5f-e3831947bf68].
Logs are available at [https://console.cloud.google.com/gcr/builds/4b91137b-fee9-445e-bf5f-e3831947bf68?project=612745165979].


#### Now create the other Kubeflow Pipelines Components

- Prepare Dataset component: 

In [8]:
%%bash -s "$GCP_PROJECT_ID"

# -------------------------------------
#     Define a Dockerfile, Build a  
#       Docker image to package 
#   the Kubeflow Pipelines Operation 
# -------------------------------------

cat > ./Dockerfile <<- "EOF"
FROM tensorflow/tensorflow:1.14.0-py3
RUN pip install --quiet tensorflow==2.0.0-beta1 Pillow 
ADD ./prepare_dataset.py /ml/prepare_dataset.py
WORKDIR /ml

ENTRYPOINT ["python", "/ml/prepare_dataset.py"]

EOF

# -------------------------------------
#       Build a Docker Image on GCP 
#         using the gcloud tool
# -------------------------------------

gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}
gcloud builds submit --tag gcr.io/$1/prepare_dataset:latest .


----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "e32a62b1-cab5-4224-b793-d74441cd3d79"

FETCHSOURCE
Fetching storage object: gs://wl-tex10-kfp-001_cloudbuild/source/1562961075.23-7235fb39ffea4ce59ed4730a711e5aa0.tgz#1562961098513370
Copying gs://wl-tex10-kfp-001_cloudbuild/source/1562961075.23-7235fb39ffea4ce59ed4730a711e5aa0.tgz#1562961098513370...
/ [1 files][178.3 MiB/178.3 MiB]                                                
Operation completed over 1 objects/178.3 MiB.                                    
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker

Step 1/5 : FROM tensorflow/tensorflow:1.14.0-py3
1.14.0-py3: Pulling from tensorflow/tensorflow
5b7339215d1d: Already exists
14ca88e9f672: Already exists
a31c3b1caad4: Already exists
b054a26005b7: Already exists
8832e3773578: Pulling fs layer
5e671b828b2a: Pulling fs layer
2b940936f993: Pulling fs layer
016724bbd2c9: Pulling fs layer
5bd1cb597025: Pulling fs layer


Activated service account credentials for: [kubeflow-user@wl-tex10-kfp-001.iam.gserviceaccount.com]
Creating temporary tarball archive of 1502 file(s) totalling 292.4 MiB before compression.
Uploading tarball of [.] to [gs://wl-tex10-kfp-001_cloudbuild/source/1562961075.23-7235fb39ffea4ce59ed4730a711e5aa0.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/wl-tex10-kfp-001/builds/e32a62b1-cab5-4224-b793-d74441cd3d79].
Logs are available at [https://console.cloud.google.com/gcr/builds/e32a62b1-cab5-4224-b793-d74441cd3d79?project=612745165979].


- Train the Pix2Pix model:

**WIP** ADD GPU !


In [9]:
%%bash -s "$GCP_PROJECT_ID"

# -------------------------------------
#     Define a Dockerfile, Build a  
#       Docker image to package 
#   the Kubeflow Pipelines Operation 
# -------------------------------------

cat > ./Dockerfile <<- "EOF"
FROM tensorflow/tensorflow:1.14.0-py3
RUN pip install --quiet tensorflow==2.0.0-beta1 Pillow 
ADD ./train_pix2pix.py /ml/train_pix2pix.py
WORKDIR /ml

ENTRYPOINT ["python", "/ml/train_pix2pix.py"]

EOF

# -------------------------------------
#       Build a Docker Image on GCP 
#         using the gcloud tool
# -------------------------------------

gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}
gcloud builds submit --tag gcr.io/$1/train_pix2pix:latest .


----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "adc07b1a-85bd-419c-949a-1d5df624692e"

FETCHSOURCE
Fetching storage object: gs://wl-tex10-kfp-001_cloudbuild/source/1562961228.22-d216aed74d1d4882a5cead76f238102a.tgz#1562961251813188
Copying gs://wl-tex10-kfp-001_cloudbuild/source/1562961228.22-d216aed74d1d4882a5cead76f238102a.tgz#1562961251813188...
/ [1 files][178.3 MiB/178.3 MiB]                                                
Operation completed over 1 objects/178.3 MiB.                                    
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker

Step 1/5 : FROM tensorflow/tensorflow:1.14.0-py3
1.14.0-py3: Pulling from tensorflow/tensorflow
5b7339215d1d: Already exists
14ca88e9f672: Already exists
a31c3b1caad4: Already exists
b054a26005b7: Already exists
8832e3773578: Pulling fs layer
5e671b828b2a: Pulling fs layer
2b940936f993: Pulling fs layer
016724bbd2c9: Pulling fs layer
5bd1cb597025: Pulling fs layer


Activated service account credentials for: [kubeflow-user@wl-tex10-kfp-001.iam.gserviceaccount.com]
Creating temporary tarball archive of 1504 file(s) totalling 292.7 MiB before compression.
Uploading tarball of [.] to [gs://wl-tex10-kfp-001_cloudbuild/source/1562961228.22-d216aed74d1d4882a5cead76f238102a.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/wl-tex10-kfp-001/builds/adc07b1a-85bd-419c-949a-1d5df624692e].
Logs are available at [https://console.cloud.google.com/gcr/builds/adc07b1a-85bd-419c-949a-1d5df624692e?project=612745165979].


- Test the Pix2Pix model: Generate/Translate images from the Test dataset.

**WIP** ADD GPU !

In [10]:
%%bash -s "$GCP_PROJECT_ID"

# -------------------------------------
#     Define a Dockerfile, Build a  
#       Docker image to package 
#   the Kubeflow Pipelines Operation 
# -------------------------------------

cat > ./Dockerfile <<- "EOF"
FROM tensorflow/tensorflow:1.14.0-py3
RUN pip install --quiet tensorflow==2.0.0-beta1 Pillow 
ADD ./test_pix2pix.py /ml/test_pix2pix.py
WORKDIR /ml

ENTRYPOINT ["python", "/ml/test_pix2pix.py"]

EOF

# -------------------------------------
#       Build a Docker Image on GCP 
#         using the gcloud tool
# -------------------------------------

gcloud auth activate-service-account --key-file=${GOOGLE_APPLICATION_CREDENTIALS}
gcloud builds submit --tag gcr.io/$1/test_pix2pix:latest .


----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "7a1d4396-a5fd-408e-bc39-358c01d4650a"

FETCHSOURCE
Fetching storage object: gs://wl-tex10-kfp-001_cloudbuild/source/1562961378.86-214d7c4cbbeb4ed2b2e3ad18943336af.tgz#1562961402082950
Copying gs://wl-tex10-kfp-001_cloudbuild/source/1562961378.86-214d7c4cbbeb4ed2b2e3ad18943336af.tgz#1562961402082950...
| [1 files][178.3 MiB/178.3 MiB]                                                
Operation completed over 1 objects/178.3 MiB.                                    
BUILD
Already have image (with digest): gcr.io/cloud-builders/docker

Step 1/5 : FROM tensorflow/tensorflow:1.14.0-py3
1.14.0-py3: Pulling from tensorflow/tensorflow
5b7339215d1d: Already exists
14ca88e9f672: Already exists
a31c3b1caad4: Already exists
b054a26005b7: Already exists
8832e3773578: Pulling fs layer
5e671b828b2a: Pulling fs layer
2b940936f993: Pulling fs layer
016724bbd2c9: Pulling fs layer
5bd1cb597025: Pulling fs layer


Activated service account credentials for: [kubeflow-user@wl-tex10-kfp-001.iam.gserviceaccount.com]
Creating temporary tarball archive of 1506 file(s) totalling 293.0 MiB before compression.
Uploading tarball of [.] to [gs://wl-tex10-kfp-001_cloudbuild/source/1562961378.86-214d7c4cbbeb4ed2b2e3ad18943336af.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/wl-tex10-kfp-001/builds/7a1d4396-a5fd-408e-bc39-358c01d4650a].
Logs are available at [https://console.cloud.google.com/gcr/builds/7a1d4396-a5fd-408e-bc39-358c01d4650a?project=612745165979].


**WIP Investigate : kfp.compiler.build_docker_image ?**

!gsutil cp ./download_dataset.py gs://wl-tex10-kfp-001/docker/download_dataset.py
!gsutil cp ./Dockerfile gs://wl-tex10-kfp-001/docker/Dockerfile


kfp.compiler.build_docker_image(staging_gcs_path='gs://wl-tex10-kfp-001/staging', 
                                target_image='gcr.io/{}/test:latest'.format(GCP_PROJECT_ID), 
                                dockerfile_path='gs://wl-tex10-kfp-001/docker/Dockerfile', 
                                timeout=600, namespace='kubeflow')

In [11]:
#### WIP : TEST A SINGLE COMPONENT PIPELINE

In [12]:
# ------------------------------------- 
# Create Kubeflow Pipelines Operations
#        from Docker Images
# -------------------------------------

def download_op(fname=FILE_NAME, origin=URL, 
                cachedir=KERAS_CACHE_DIR, cachesubdir='datasets'):
    
    return kfp.dsl.ContainerOp(name="Download Dataset",
                               image='gcr.io/{}/download_dataset:latest'.format(GCP_PROJECT_ID),
                               command=['python', '/ml/download_dataset.py'],
                               arguments=['--fname', fname,
                                          '--origin', origin,
                                          '--cachedir', cachedir,
                                          '--cachesubdir', cachesubdir ],
                               file_outputs = {'outputdir': '/output.txt'}
                              )


def prepare_dataset_op(pathimg, pathimgsubdir, op_name):

    return kfp.dsl.ContainerOp(name=op_name,
                               image='gcr.io/{}/prepare_dataset:latest'.format(GCP_PROJECT_ID),
                               command=['python', '/ml/prepare_dataset.py'],
                               arguments=['--pathimg', pathimg,
                                          '--pathimgsubdir', pathimgsubdir ],
                               file_outputs = {'outputdir': '/output.txt'}
                              )


def train_op(pathdataset, epochs, pathmodel):

    return kfp.dsl.ContainerOp(name="Train Pix2Pix model",
                               image='gcr.io/{}/train_pix2pix:latest'.format(GCP_PROJECT_ID),
                               command=['python', '/ml/train_pix2pix.py'],
                               arguments=['--dataset', pathdataset,
                                          '--model', pathmodel,
                                          '--epochs', epochs ],
                               file_outputs = {'outputdir': '/output.txt'}
                              )


def test_op(pathdataset, pathoutput, pathmodel):

    return kfp.dsl.ContainerOp(name="Test Pix2Pix",
                               image='gcr.io/{}/test_pix2pix:latest'.format(GCP_PROJECT_ID),
                               command=['python', '/ml/test_pix2pix.py'],
                               arguments=['--dataset', pathdataset,
                                          '--output', pathoutput,
                                          '--model', pathmodel ],
                               file_outputs = {'outputdir': '/output.txt'}
                              )

In [13]:
# ------------------------------------- 
#  Build the pix2pix Pipeline Function
# -------------------------------------

@dsl.pipeline(
    name='TEST pipeline',
    description='A pipeline to download and prepare the dataset'
)

def pix2pix(
    
    ## -- Download Dataset Kubeflow Pipeline component parameters (with default values)
    origin = dsl.PipelineParam('origin', value=URL),
    fname = dsl.PipelineParam('fname', value=FILE_NAME),
    cachedir = dsl.PipelineParam('cachedir', value=KERAS_CACHE_DIR), # on Kubeflow GKE/NFS 
    cachesubdir = dsl.PipelineParam('cachesubdir', value="datasets"),
    
    ## -- Prepare Dataset Kubeflow Pipeline component parameters (with default values)
    pathimgsubdirtrain = dsl.PipelineParam('pathimgsubdirtrain', value="train"),
    pathimgsubdirtest = dsl.PipelineParam('pathimgsubdirtest', value="test"),
    
    ## -- Train Pix2Pix Kubeflow Pipeline component parameters (with default values)
    epochs = dsl.PipelineParam('epochs', value="1"),
    model = dsl.PipelineParam('epochs', value=MODEL),
    
    ## -- Test Pix2Pix Kubeflow Pipeline component parameters (with default values)
    output = dsl.PipelineParam('output', value=PIX2PIX_OUTPUTS)
):
    
    
    # Passing pipeline parameters as operation arguments (Returns a dsl.ContainerOp class instance)
    download_task = download_op(fname, origin, cachedir, cachesubdir) \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
    
    
    prepare_train_task = prepare_dataset_op(pathimg=download_task.output, 
                                            pathimgsubdir=pathimgsubdirtrain,
                                            op_name="Prepare Train Dataset") \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
        

    prepare_test_task = prepare_dataset_op(pathimg=download_task.output,
                                           pathimgsubdir=pathimgsubdirtest,
                                           op_name="Prepare test Dataset") \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
    
    ## WIP : add GPU
    train_task = train_op(pathdataset=prepare_train_task.output,
                          pathmodel=model,
                          epochs=epochs) \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))
    
     ## WIP : add GPU
    test_task = test_op(pathdataset=prepare_test_task.output, 
                        pathoutput=output,
                        pathmodel=train_task.output) \
                                .add_volume(k8s_client.V1Volume(name='workdir', 
                                                                persistent_volume_claim=k8s_client.V1PersistentVolumeClaimVolumeSource(claim_name='nfs'))) \
                                .add_volume_mount(k8s_client.V1VolumeMount(mount_path=NFS_MOUNT, name='workdir'))

In [14]:
# -------------------------------------
#       Compile the Pipeline 
# -------------------------------------

pipeline_filename = pix2pix.__name__ + '.pipeline.tar.gz'
compiler.Compiler().compile(pipeline_func=pix2pix, 
                            package_path=pipeline_filename)


In [15]:
# ------------------------------------- 
#           Create (or reuse) a    
#      Kubeflow Pipeline Experiment
# -------------------------------------

EXPERIMENT_NAME = "TEST - Pix2Pix"   ## Customize Name
client = kfp.Client()

try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)

# -------------------------------------
#              Optional : 
# Specify/Overwrite pipeline arguments 
#     default values for execution
# -------------------------------------

#arguments = {'epochs': 1 }# Change to 200 for a full training 

# -------------------------------------
#       Submit a pipeline run
# -------------------------------------
run_name = pix2pix.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename)
#run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)


In [ ]:
!ls /mnt/nfs/data/datasets/facades
!ls /mnt/nfs/data/models

#### Cleaning
